# **WEEK 2 Analysis**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


plt.style.use("seaborn-v0_8")


**For this week's analysis, I will use AAPL asset.**

In [3]:
apple_data = yf.download('AAPL' , start = '2015-01-01', end = '2025-01-01',auto_adjust=False)

[*********************100%***********************]  1 of 1 completed




FEATURE CONSTRUCTION

In [4]:
apple_price = apple_data['Adj Close']
feature = pd.DataFrame()
feature['r_t'] = np.log(apple_data['Adj Close']/apple_data['Adj Close'].shift(1))
feature['r_t-1'] = feature.r_t.shift(1)
feature['20-day_rolling_mean'] = apple_price.rolling(window=20).mean()
feature['20-day_rolling_volatility'] = feature['r_t'].rolling(window=20).std()
feature['5-day_momentum'] = feature['r_t'].rolling(window=5).mean()

## **NAIVE BASELINES**

1. ZERO PREDICTOR

In [20]:
apple_price.tail()
type(apple_price.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [ ]:
date = input("Input the date in YYYY-MM-DD") 
target_date = pd.to_datetime(date)  #date is in the format of the string, whereas target date is now in the format of DatetimeIndex in pandas.
prev_day_price = apple_price.shift(1)   #ek din shift kara

#try and except use kara, ki agar market band ho toh kaam ayega
try:
    predicted_price = prev_day_price.loc[target_date]       
    predicted_return = 0
    print(f'predicted price = {predicted_price.item()}')
    print(f'predicted return = {predicted_return}')
except KeyError:
    print(f"Error: The date '{date}' was not found. It might be a weekend or outside your data range.")

predicted price = 257.853759765625
predicted return = 0


2. Rolling mean predictor - 5-day mean of returns

In [25]:
date = input('Input the date as YYYY-MM-DD')
target_date = pd.to_datetime(date)
prev_momentum = feature['5-day_momentum'].shift(1)

try:
    predicted_return = prev_momentum.loc[target_date]
    print(f"predicted return = {predicted_return}")
    
except KeyError:
    print(f"Error: The date '{date}' was not found. It might be a weekend or outside your data range.")


predicted return = 0.008654951791642397
